In [88]:
import json
import pickle

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import warnings

%matplotlib inline
warnings.filterwarnings('ignore')

# Loading

In [3]:
with open('../utility_dataframes/new_names.pkl', 'rb') as file:
    new_names = pickle.load(file)

In [55]:
excel_df = pd.read_csv('../ready_dataframes/excel_df_correct_names.csv', index_col=0)

In [56]:
csv_df = pd.read_csv('../ready_dataframes/boarding_data_processed.csv', index_col=0)

In [57]:
excel_df['Name'] = excel_df['FirstName'] + ' ' + excel_df['LastName']
csv_df['Name'] = csv_df['FirstName'] + ' ' + csv_df['LastName']

In [14]:
unique_excel_names = set(excel_df['Name'])
new_names = []

for i in tqdm(range(csv_df.shape[0])):
    if csv_df.iloc[i, 14] not in unique_excel_names:
        new_names.append(i)

100%|██████████| 155147/155147 [00:01<00:00, 115045.15it/s]


# Add lack columns

In [58]:
excel_df['BirthDate'] = np.nan
excel_df['Document'] = np.nan
excel_df['BonusProgrammNumber'] = np.nan

In [59]:
csv_df['DepartureCountry'] = np.nan
csv_df['DepartureCity'] = np.nan
csv_df['ArrivalCountry'] = np.nan
csv_df['AirlineName'] = np.nan
csv_df['LoyaltyStatus'] = np.nan
csv_df['BonusProgrammNumber'] = np.nan

# Deal with missing data in csv

## Departure & Arrival Airports

In [60]:
fly_root = pd.read_csv('../utility_dataframes/fly_root.csv', names=['Date', 'Number', 'Departure', 'Arrival', 'Status'])

In [89]:
fly_root

,Departure,Arrival
Number,,
AF10,CDG,JFK
AF1000,CDG,MAD
AF1001,MAD,CDG
AF1004,CDG,FCO
AF1005,FCO,FCO
...,...,...
VN931,LPQ,REP
VN942,RGN,SGN
VN943,SGN,RGN


In [61]:
fly_root = fly_root.groupby("Number")['Departure', 'Arrival'].max()

In [62]:
def extract_departure_arrival_airport(number):
    return tuple(fly_root.loc[number])

In [63]:
csv_df['DepartureAirport'], csv_df['ArrivalAirport'] = \
zip(*csv_df['FlightNumber'].map(extract_departure_arrival_airport))

## Departure & Arrival Locations

In [11]:
airports_df = pd.read_csv('../utility_dataframes/airports_df.csv', index_col=0)

In [12]:
airports_df.set_index('code', inplace=True)
airports_df

,city,country
code,,
AAA,Anaa,French Polynesia
AAE,El Tarf,Algeria
AAL,Norresundby,Denmark
AAM,Mala Mala,South Africa
AAN,Ayn al Faydah,United Arab Emirates
...,...,...
YZY,Zhangye,China
YUS,Yushu,China
BPE,Bagan,Burma


In [90]:
airports_df.drop('AAA')

,city,country
code,,
AAE,El Tarf,Algeria
AAL,Norresundby,Denmark
AAM,Mala Mala,South Africa
AAN,Ayn al Faydah,United Arab Emirates
AAQ,Novorossiysk,Russia
...,...,...
YZY,Zhangye,China
YUS,Yushu,China
BPE,Bagan,Burma


In [13]:
def extract_city_country(code):
    return tuple(airports_df.loc[code])

In [64]:
csv_df['DepartureCity'], csv_df['DepartureCountry'] = \
zip(*csv_df['DepartureAirport'].map(extract_city_country))

KeyboardInterrupt: 

In [ ]:
csv_df['ArrivalCity'], csv_df['ArrivalCountry'] = \
zip(*csv_df['ArrivalAirport'].map(extract_city_country))

In [ ]:
csv_df['DepartureCity'] = csv_df['DepartureCity'].apply(lambda x: x.lower())
csv_df['DepartureCountry'] = csv_df['DepartureCountry'].apply(lambda x: x.lower())
csv_df['ArrivalCity'] = csv_df['ArrivalCity'].apply(lambda x: x.lower())
csv_df['ArrivalCountry'] = csv_df['ArrivalCountry'].apply(lambda x: x.lower())

## Airline Name

In [28]:
airline_names = {
    'SU': 'aeroflot',
    'OK': 'chech airlines',
    'KE': 'korean air',
    'UX': 'air europa',
    'CZ': 'china southern airlines',
    'AF': 'air france',
    'KL': 'klm',
    'VN': 'vietnam airlines',
    'RO': 'tarom',
    'AZ': 'alitalia',
    'CI': 'china airlines',
    'MF': 'xiamen air',
    'ME': 'middle east airlines',
    'MU': 'china eastern airlines',
    'AR': 'argentinas',
    'GA': 'garudas indonesia',
    'DL': 'delta airlines',
    'KQ': 'kenya airways',
    'SV': 'saudi arabian airlines',
    'AM': 'aeromexico'
}

def convert_airline_name(name, airline_names=airline_names):
    return airline_names.get(name[:2], np.nan)

In [29]:
csv_df['AirlineName'] = csv_df['FlightNumber'].map(convert_airline_name)

In [31]:
csv_df.to_csv('../ready_dataframes/boarding_data_fix.csv')

In [65]:
# csv_df = pd.read_csv('../ready_dataframes/boarding_data_fix.csv', index_col=0)

# Make MultiIndex

Excel

In [66]:
excel_df.drop(['FirstName', 'LastName'], axis=1, inplace=True)

In [67]:
excel_df_multi = excel_df.set_index(['Name', 'FlightNumber', 'Date']).sort_index()

CSV

In [68]:
csv_df['Gender'] = csv_df['Gender'].map(lambda x: 'M' if x == 'male' else 'F')

In [69]:
csv_df.drop(['FirstName', 'SecondName', 'LastName', 'TicketNumber', 'Baggage', 'CodeShare', 'BookingCode'], 
            axis=1, inplace=True)

In [70]:
csv_df_multi = csv_df.set_index(['Name', 'FlightNumber', 'Date']).sort_index()

# Repeated index

In [71]:
repeat_index = csv_df_multi.index & excel_df_multi.index

In [72]:
csv_uniq = csv_df_multi.drop(repeat_index, axis=0)

In [73]:
full_concat = pd.concat((excel_df_multi, csv_uniq)).sort_index()

Store for future

In [74]:
full_concat.to_csv('../ready_dataframes/excel_with_csv.csv')

In [75]:
full_concat_reset_index = full_concat.reset_index()

In [76]:
full_concat_reset_index = pd.read_csv('../ready_dataframes/excel_with_csv.csv')

In [31]:
def agg_birtdates(x):
    list_birthdates = list(set([date for date in x if date is not np.nan]))
    if list_birthdates:
        return list_birthdates.pop()
    return np.nan

In [32]:
birth_dates = full_concat.reset_index().groupby('Name')['BirthDate'].agg(agg_birtdates)

In [33]:
with open('../utility_dataframes/birth_dates.pkl', 'wb') as f:
    pickle.dump(birth_dates, f)

In [77]:
full_concat_reset_index['BirthDate'] = full_concat_reset_index['Name'].map(lambda x: birth_dates.get(x, np.nan))

In [35]:
documents = full_concat_reset_index.groupby('Name')['Document'].agg(agg_birtdates)

In [36]:
with open('../utility_dataframes/documents.pkl', 'wb') as f:
    pickle.dump(documents, f)

In [78]:
full_concat_reset_index['Document'] = full_concat_reset_index['Name'].map(lambda x: documents.get(x, np.nan))

In [38]:
genders = full_concat_reset_index.groupby('Name')['Gender'].agg(agg_birtdates)

In [39]:
with open('../utility_dataframes/genders.pkl', 'wb') as f:
    pickle.dump(genders, f)

In [40]:
def split_name(name):
    return tuple(name.split(' '))

In [79]:
full_concat_reset_index['FirstName'], full_concat_reset_index['LastName'] = \
zip(*full_concat_reset_index['Name'].map(split_name))

In [42]:
def delete_soft_sign(name):
    pos = name.find("'")
    if pos == -1:
        return name
    return name[:pos] + name[pos+1:]

In [80]:
def replace_chars(name):
    pos_x = name.find('x')
    if pos_x != -1:
        name = name[:pos_x] + 'ks' + name[pos_x+1:]
    
    pos_ia = name.find('ia')
    if pos_ia != -1:
        name = name[:pos_ia] + 'ya' + name[pos_ia+2:]
    
    pos_iu = name.find('iu')
    if pos_iu != -1:
        name = name[:pos_iu] + 'yu' + name[pos_iu+2:]
    
    pos_yi = name.find('yi')
    if pos_yi != -1:
        name = name[:pos_yi] + 'yy' + name[pos_yi+2:]
        
    pos_tc = name.find('tc')
    if pos_tc != -1:
        name = name[:pos_tc] + 'ts' + name[pos_tc+2:]
        
    pos_io = name.find('io')
    if pos_io != -1:
        name = name[:pos_io] + 'yo' + name[pos_io+2:]
        
    pos_ii = name.find('ii')
    if pos_ii != -1:
        name = name[:pos_ii] + 'iy' + name[pos_ii+2:]
        
    pos_ja = name.find('ja')
    if pos_ja != -1:
        name = name[:pos_ja] + 'ya' + name[pos_ja+2:]
        
    pos_ij = name.find('ij')
    if pos_ij != -1:
        name = name[:pos_ij] + 'iy' + name[pos_ij+2:]
        
    pos_ju = name.find('ju')
    if pos_ju != -1:
        name = name[:pos_ju] + 'yu' + name[pos_ju+2:]
        
    return name

In [81]:
full_concat_reset_index['LastName'] = full_concat_reset_index['LastName'].map(delete_soft_sign)

In [82]:
full_concat_reset_index['LastName'] = full_concat_reset_index['LastName'].map(replace_chars)

In [83]:
full_concat_reset_index['FirstName'] = full_concat_reset_index['FirstName'].map(delete_soft_sign)
full_concat_reset_index['FirstName'] = full_concat_reset_index['FirstName'].map(replace_chars)

In [84]:
full_concat_reset_index['Name'] = full_concat_reset_index['FirstName'] + ' ' + full_concat_reset_index['LastName'] 

In [85]:
full_concat = full_concat_reset_index.set_index(['Name', 'FlightNumber', 'Date']).sort_index()

In [86]:
full_concat

AirlineName ArrivalAirport  \
Name          FlightNumber Date                                                 
adel afanasev CZ3383       2017-03-11  china southern airlines            NNY   
              CZ6478       2017-07-21  china southern airlines            CGO   
              SU1457       2017-01-21                 aeroflot            SVO   
adel ageev    CZ3982       2017-11-02  china southern airlines            CGO   
              DL1508       2017-11-27           delta airlines            ATL   
...                                                        ...            ...   
zlata zykova  AF4921       2017-03-04               air france            CDG   
              CZ3547       2017-03-30  china southern airlines            SHA   
              DL3901       2017-12-01           delta airlines            VLD   
              DL7424       2017-12-12           delta airlines            DTW   
              MU6118       2017-05-06   china eastern airlines            NGB   

                                       ArrivalCity ArrivalCountry   BirthDate  \
Name          FlightNumber Date                                                 
adel afanasev CZ3383       2017-03-11      nanyang          china         NaN   
              CZ6478       2017-07-21    zhengzhou          china  1976-01-16   
              SU1457       2017-01-21   zelenograd         russia  1976-01-16   
adel ageev    CZ3982       2017-11-02    zhengzhou          china  1972-05-23   
              DL1508       2017-11-27   atlanta ga  united states  1972-05-23   
...                                            ...            ...         ...   
zlata zykova  AF4921       2017-03-04        paris         france         NaN   
              CZ3547       2017-03-30     shanghai          china         NaN   
              DL3901       2017-12-01  valdosta ga  united states         NaN   
              DL7424       2017-12-12   detroit mi  united states         NaN   
              MU6118       2017-05-06       ningbo          china         NaN   

                                       BonusProgrammNumber DepartureAirport  \
Name          FlightNumber Date                                               
adel afanasev CZ3383       2017-03-11                  NaN              CAN   
              CZ6478       2017-07-21                  NaN              SZX   
              SU1457       2017-01-21                  NaN              KEJ   
adel ageev    CZ3982       2017-11-02                  NaN              SYX   
              DL1508       2017-11-27                  NaN              SRQ   
...                                                    ...              ...   
zlata zykova  AF4921       2017-03-04                  NaN              SVO   
              CZ3547       2017-03-30                  NaN              CAN   
              DL3901       2017-12-01                  NaN              ATL   
              DL7424       2017-12-12                  NaN              FWA   
              MU6118       2017-05-06                  NaN              HYN   

                                               DepartureCity DepartureCountry  \
Name          FlightNumber Date                                                 
adel afanasev CZ3383       2017-03-11              guangzhou            china   
              CZ6478       2017-07-21               shenzhen            china   
              SU1457       2017-01-21               kemerovo           russia   
adel ageev    CZ3982       2017-11-02                  sanya            china   
              DL1508       2017-11-27  sarasota/bradenton fl    united states   
...                                                      ...              ...   
zlata zykova  AF4921       2017-03-04                 moscow           russia   
              CZ3547       2017-03-30              guangzhou            china   
              DL3901       2017-12-01             atlanta ga    united states   
              DL7424    

In [87]:
full_concat.to_csv('../ready_dataframes/excel_with_csv.csv')